In [9]:
# !hdfs dfs -get hdfs://harunava/user/leon.kepler/clm/data.json ./data.json

In [1]:
import json
with open("data.json","r") as f:
    data = json.load(f)

In [2]:
def format_record(record):
    return f"claim: {record['claim']}, notes: {record['notes']}"
   
str_data = [format_record(record) for record in data]

In [3]:
for i in range(10):
    print(str_data[i])

claim: , notes: This is a satire. It may be offensive through linking china to covid, but it is not misinformation as it is humour. 
claim: , notes: The video describes the Alcea rugosa water extract preparation for the treatment of stomach ulcer (collect petals, cover with cold water, leave it overnight) as well as the treatment instructions (take one tablespoon before food for 10-15 days). 
Traditional herbal medicine can not treat stomach ulcers since it is a severe health problem requiring drug treatment (antibiotics in case ulcers were caused by the infection with bacteria Helicobacter pylori).

claim: , notes: DESCRIPTION: the claim, made by candace owens, is that because black Americans commit more crime, that is why they are killed by the police. Her methodology is flawed.  “These quantities can differ enormously: When officers encounter many more white civilians (due to whites’ majority status, for example), the proportion of killings involving black civilians can be small, ev

In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='1'

In [12]:
import cudf
from cudf import Series
from cudf import DataFrame

from cuml.feature_extraction.text import TfidfVectorizer

In [13]:
import numpy as np
import pandas as pd
import warnings


pd.set_option('display.max_colwidth', 1000000)
warnings.filterwarnings('ignore')

In [17]:
def join_df(path, df_lib=cudf):
    data = df_lib.DataFrame()
    temp = df_lib.read_csv(path)
    # temp = temp[temp.lang=='en']
    data = df_lib.concat([data,temp])
    # for file in os.listdir(path):
    #     print(f"In path : {path}{file}")
    #     temp = df_lib.read_csv(path+file)
    #     temp = temp[temp.lang=='en']
    #     data = df_lib.concat([data,temp])
    return data

In [18]:
df = join_df('/opt/tiger/workspace/LLM/data.csv')
tweets = Series(df['text'])
len(tweets)

176722

In [19]:
vec = TfidfVectorizer(stop_words='english')

tfidf_matrix = vec.fit_transform(tweets)
tfidf_matrix.shape

(176722, 288685)

In [23]:
from cuml.common.sparsefuncs import csr_row_normalize_l2


def efficient_csr_cosine_similarity(query, tfidf_matrix, matrix_normalized=False):
    query = csr_row_normalize_l2(query, inplace=False)
    if not matrix_normalized:
        tfidf_matrix = csr_row_normalize_l2(tfidf_matrix, inplace=False)
    
    return tfidf_matrix.dot(query.T)


def document_search(text_df, query, vectorizer, tfidf_matrix, top_n=10):
    query_vec = vectorizer.transform(Series([query]))
    similarities = efficient_csr_cosine_similarity(query_vec, tfidf_matrix, matrix_normalized=True)
    similarities = similarities.todense().reshape(-1)
    best_idx = similarities.argsort()[-top_n:][::-1]
    
    pp = cudf.DataFrame({
        'text': text_df['text'].iloc[best_idx],
        'similarity': similarities[best_idx]
    })
    return pp

In [31]:
text = "من ام الفعير يلعن ابوك لابو طيزر عرضك"
document_search(df, text, vec, tfidf_matrix)

,text,similarity
106515,"claim: The video claims that a certain healthy diet can prevent from cancer. The diet must depend on vegetables and fruits.\nIt stresses the importance of eating beetroot and its juice which has many benefits. Mixing vegetables and drinking their juice helps a, notes: يكتسب الشمندر لونه الرائع من أحد أنواع مضادات الأكسدة (Antioxidants) الهامة المتواجدة فيه والمدعوة بالبيتالينات، وهذا النوع من مضادات الأكسدة بشكل خاص قد يحمي الجسم من بعض أنواع الخلايا السرطانية. كما يعتقد الباحثون أن البيتالينات تعمل على الكشف في داخل الجسم عن كافة الخلايا غير المستقرة أو السرطانية لتدمرها. السرطان هو ثاني سبب رئيسي للوفاة في الولايات المتحدة، لكن التقديرات تشير إلى أنه يمكن الوقاية مما يصل إلى ثلث الحالات عن طريق اتباع نظام غذائي والتغذية وحدها. ما هي الوصفة؟ يوصي العديد من الخبراء بالأكل من خيرات الأرض. تشير عقود من البحث إلى أن أفضل نظام غذائي للوقاية من السرطان هو كل ما يتضمن النباتات. وهذا يعني الكثير من الفواكه والخضراوات والبقوليات، أو القليل من اللحوم أو غيرها من المنتجات الحيوانية.\nقول اختصاصي النُّظم الغذائية في برنامج Mayo Clinic للحياة الصحية أنجي مراد، اختصاصية تغذية مسجلة ومرخصة: إنه على الرغم من أن الدراسات الجديدة قد احتلت العناوين الرئيسية، فإن الجزء الأكبر من البحث لا يزال يدعم تناول كميات أقل من اللحوم. وتقول: ""هناك الكثير من الأدلة للتحرك نحو نظام غذائي نباتي"".\n",0.255338
109396,"claim: Claim: the video says that Henry Kissinger has called for an end to Russia-Ukraine war to prevent a Russian-Chinese alliance as the world will be preoccupied with a Middle East upcoming war between Iran and Israel which will also involve Saudi Arabia , E, notes: ما جاء في نقل الرجل لتوقعات وزير الخارجية الامريكي \n السابق هنري كيسنجر في لقاء اجرته معه صحيفة الصنداي تايم حيث اكد ان حرب بين اسرائيل وايران قادمة ولكن كيسنجر لم يحدد اين ستقع هذه الحرب هل ستكون على الارض اللبنانية ام السورية ام الايرانية\nالخبر صحيح وقد ترجم المتحدث في الفيدبو نعض مقاطع من اللقاء الصحفي.",0.245030
113445,"claim: Claim: The video claims that there are food supplements that are natural and healthy, and cures the body from illnesses. Such as Spirulina and Reishi Mushrooms, and DXN supplements which are very important for health., notes: \nعلى الرغم من فوائد الفطر الريشي إلا أنه ذو أضرار خاصةً على الفئات الآتية:\nالأفراد الذين يعانون من انخفاض ضغط الدم.\nالأشخاص الذين يتناولون الأدوية من أجل العلاج من ارتفاع ضغط الدم، أو السكري، أو اضطرابات الجهاز المناعي.\nكما أن استهلاكه لمدة تزيد عن ستة شهور قد تسبب رد فعل تحسسيّ لدى بعض الأشخاص، وينتج عن ذلك مجموعة من الأعراض من أبرزها ما يأتي:\nجفاف في الفم والحلق و ممرات الأنف.\nالشعور بالدوخة.\nالمعاناة من الحكّة.\nالإصابة بالطفح الجلديّ.\nاما سبيرولينا فهي:\nالسبيرولينا هي نوع من أنواع الطحالب لونها أخضر مائل للزرقة، وتنمو في المحيطات والبحيرات المالحة في المناخات شبه الاستوائية.\nتعد السبيرولينا من الأغذية الخارقة بسبب غناها بالأصباغ النباتية، وقدرتها العالية على تنظيم عملية التمثيل الضوئي، ما جعلها أحد المكونات المشهورة في المكملات الغذائية.\n \nفيما يتعلق بالمكملات الغذائية من دكسن هناك تحذير من مؤسسة الغذاء والدواء الاردنية الرسمية بهذا الخصوص.\n",0.239793
113496,"claim: Claim: The video claims that Iron mixed with vitamin c and some herbs has a strong effect on the body. It provides the body with the necessary health supplements., notes: \nيُعدّ فيتامين ج: Vitamin C) أ\nمن الفيتامينات الذائبة في الماء، ويعمل في الجسم كمُضادّ للأكسدة؛ حيث يحمي من الأضرار الناتجة عن الجُذور الحُرّة -وهي مُركّباتٌ تنتج عندما يتعرّض الجسم للإشعاعات أو دخان التبغ- وقد تُرتبط بزيادة خطر الإصابة بالعديد من الأمراض، ويحتاج الجسم هذا الفيتامين لصناعة الكولاجين، وغيرها من الفوائد، ومن الجدير بالذكر أنّ الجسم لا يُصنّع هذا الفيتامين، ولذلك فإنّه يحتاج لتناوله من مصادره الغذائيّة، فهو يتوفر في العديد من الخضار والفاكهة، أو يمكن تناوله كمكمل غذائي على شكل حبوبٍ قابلة للمضغ أو كبسولات.0 seconds of 0 secondsVolume 0% أما الحديد فهو من المعادن الموجودة في الجسم والتي يحتاجها للقيام بالعديد من الوظائف، فهو من مكونات الهيموغلوبين الذي يُساهم في نقل الأكسجين من الرئتين إلى باقي أجزاء الجسم، وقد ينجم عن عدم تناول كميات كافية من م